<a href="https://colab.research.google.com/github/Sahil-Chhabra-09/Book-Recommendation-System/blob/main/Books_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Books Recommendation System based on Cosine Similarity

In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Book Recommendation System/BooksData.xlsx")

In [ ]:
print(data.shape)
data.head()

(16559, 7)


,Unnamed: 0,Index,ID,BookTitle,Author,Genre,Summary
0,0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,2,3,986,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,3,4,1756,An Enquiry Concerning Human Understanding,David Hume,NaN,The argument of the Enquiry proceeds by a ser...
4,4,5,2080,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


In [ ]:
data.isna().sum()

Unnamed: 0       0
Index            0
ID               0
BookTitle        0
Author        2382
Genre         3718
Summary          0
dtype: int64

In [ ]:
data = data.drop(data[data['Genre'].isna()].index)
data = data.drop(data[data['Author'].isna()].index)

In [ ]:
data.isna().sum()

Unnamed: 0    0
Index         0
ID            0
BookTitle     0
Author        0
Genre         0
Summary       0
dtype: int64

In [ ]:
data.shape

(12055, 7)

In [ ]:
print(data.Genre)
data.Genre[0]   #Genre was in the form of a json object

0        {"/m/016lj8": "Roman \u00e0 clef", "/m/06nbt":...
1        {"/m/06n90": "Science Fiction", "/m/0l67h": "N...
2        {"/m/02m4t": "Existentialism", "/m/02xlf": "Fi...
4        {"/m/03lrw": "Hard science fiction", "/m/06n90...
5        {"/m/098tmk": "War novel", "/m/016lj8": "Roman...
                               ...                        
16549                      {"/m/06n90": "Science Fiction"}
16551    {"/m/01jfsb": "Thriller", "/m/02xlf": "Fiction...
16555     {"/m/01jfsb": "Thriller", "/m/02xlf": "Fiction"}
16556                         {"/m/0xdf": "Autobiography"}
16558    {"/m/02ql9": "Epistolary novel", "/m/014dfn": ...
Name: Genre, Length: 12055, dtype: object


'{"/m/016lj8": "Roman \\u00e0 clef", "/m/06nbt": "Satire", "/m/0dwly": "Children\'s literature", "/m/014dfn": "Speculative fiction", "/m/02xlf": "Fiction"}'

In [ ]:
import json
genres_cleaned = []
for i in data['Genre']:
    genres_cleaned.append(list(json.loads(i).values()))   #converts json string to dictionary which is further converted into list
genres_cleaned[0:10]

[['Roman à clef',
  'Satire',
  "Children's literature",
  'Speculative fiction',
  'Fiction'],
 ['Science Fiction',
  'Novella',
  'Speculative fiction',
  'Utopian and dystopian fiction',
  'Satire',
  'Fiction'],
 ['Existentialism', 'Fiction', 'Absurdist fiction', 'Novel'],
 ['Hard science fiction',
  'Science Fiction',
  'Speculative fiction',
  'Fantasy',
  'Fiction'],
 ['War novel', 'Roman à clef'],
 ["Children's literature",
  'Fantasy',
  'Speculative fiction',
  'Bildungsroman',
  'Fiction'],
 ['Science Fiction', 'Speculative fiction'],
 ['Science Fiction', 'Speculative fiction'],
 ['Speculative fiction', 'Fiction', 'Novel'],
 ['Science Fiction',
  'Speculative fiction',
  "Children's literature",
  'Fiction']]

In [ ]:
data['Genre'] = genres_cleaned

In [ ]:
data.Summary[0]

' Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, \'Beasts of England\'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a philosophy. The animals revolt and drive the drunken and irresponsible Mr Jones from the farm, renaming it "Animal Farm". They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal". Snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly. The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. Napoleon takes the pups from the farm dogs and trains them privately. Napoleon and Snowball struggle for leadership. When Snowball announces his plans to build a windmill, Napoleon has his dogs chase Snowball away and declares himself leader. N

In [ ]:
import re   #regular expression matching
def clean_summary(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]"," ",text)     #means any character that IS NOT a-z OR A-Z
    text = ' '.join(text.split())
    text = text.lower()
    return text

print(data['Summary'].apply(lambda x: clean_summary(x))[0])
data['Summary'] = data['Summary'].apply(lambda x: clean_summary(x))

old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy the animals revolt and drive the drunken and irresponsible mr jones from the farm renaming it animal farm they adopt seven commandments of animal ism the most important of which is all animals are equal snowball attempts to teach the animals reading and writing food is plentiful and the farm runs smoothly the pigs elevate themselves to positions of leadership and set aside special food items ostensibly for their personal health napoleon takes the pups from the farm dogs and trains them privately napoleon and snowball struggle for leadership when snowball announces his plans to build a windmill napoleon has his dogs chase snowball away and declares himself leader napoleon enacts changes to the gov

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
data['GenreString'] = data['Genre'].apply(lambda x: ' '.join(x))

In [ ]:
data['GenreString'][0]

"Roman à clef Satire Children's literature Speculative fiction Fiction"

In [ ]:
#get a combined text that includes author's name and associated genres
data["combined_text"] = data["Summary"] + " " + data["Author"] + " " + data["GenreString"]

In [ ]:
data['text_without_stopwords'] = data['combined_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
data['text_without_stopwords'][0]

"old major old boar manor farm calls animals farm meeting compares humans parasites teaches animals revolutionary song beasts england major dies two young pigs snowball napoleon assume command turn dream philosophy animals revolt drive drunken irresponsible mr jones farm renaming animal farm adopt seven commandments animal ism important animals equal snowball attempts teach animals reading writing food plentiful farm runs smoothly pigs elevate positions leadership set aside special food items ostensibly personal health napoleon takes pups farm dogs trains privately napoleon snowball struggle leadership snowball announces plans build windmill napoleon dogs chase snowball away declares leader napoleon enacts changes governance structure farm replacing meetings committee pigs run farm using young pig named squealer mouthpiece napoleon claims credit windmill idea animals work harder promise easier lives windmill violent storm animals find windmill annihilated napoleon squealer convince ani

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer()
count_matrix = cv.fit_transform(data['text_without_stopwords'])

In [ ]:
count_matrix.shape

(12055, 112000)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine = cosine_similarity(count_matrix, count_matrix)

In [ ]:
tf = TfidfVectorizer(analyzer = "word", ngram_range=(1,2), min_df=0, stop_words='english')

tfidf_matrix = tf.fit_transform(data['text_without_stopwords'])

cosine2 =  cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
def get_title_from_index(Index):
    return data[data.Index == Index].BookTitle.values[0]
def get_index_from_title(BookTitle):
    return data[data.BookTitle == BookTitle]["Index"].values[0]

def get_recommendations(book):
    book_index = get_index_from_title(book)
    similar_books = list(enumerate(cosine[book_index]))
    print(similar_books)
    sortedbooks = sorted(similar_books, key = lambda x:x[1], reverse=True)[1:]
    print(sortedbooks)
    i = 0
    for book in sortedbooks:
        if data[data.Index==book[0]].shape[0]==0:
          continue;
        title = data[data.Index==book[0]].BookTitle
        author = data[data.Index==book[0]].Author
        print(title + " by " + author)
        i = i+1
        if i>10:
            break

In [ ]:
def get_title_from_index(Index):
    return data[data.Index == Index].BookTitle.values[0]
def get_index_from_title(BookTitle):
    return data[data.BookTitle == BookTitle]["Index"].values[0]

def get_recommendations2(book):
    book_index = get_index_from_title(book)
    similar_books = list(enumerate(cosine2[book_index]))
    print(similar_books)
    sortedbooks = sorted(similar_books, key = lambda x:x[1], reverse=True)[1:]
    print(sortedbooks)
    i = 0
    for book in sortedbooks:
        if data[data.Index==book[0]].shape[0]==0:
          continue;
        title = data[data.Index==book[0]].BookTitle
        author = data[data.Index==book[0]].Author
        print(title + " by " + author)
        i = i+1
        if i>10:
            break

In [ ]:
print(get_recommendations('Dune'))

[(0, 0.03982103711470914), (1, 0.06167259073850452), (2, 0.07084859232932719), (3, 0.07404653453296435), (4, 0.07827708809363719), (5, 0.08074155783157141), (6, 0.02375048185670499), (7, 0.030923950871959635), (8, 0.04536711989222496), (9, 0.044218088868130744), (10, 0.05496647071355826), (11, 0.042123859934652744), (12, 0.06162638685966668), (13, 0.05541433889286722), (14, 0.06571340812629313), (15, 0.0797955366773321), (16, 0.035505508414349594), (17, 0.0315352919721566), (18, 0.07798585239372899), (19, 0.09201271216952676), (20, 0.06225961905955583), (21, 0.028696418087459697), (22, 0.06571741591948105), (23, 0.0436324212706176), (24, 0.0682039662594779), (25, 0.057824488163057124), (26, 0.03978103108561315), (27, 0.08216333245915813), (28, 0.05154779945813031), (29, 0.0452018475035594), (30, 0.04469678307597678), (31, 0.04598341090560416), (32, 0.0835920408296268), (33, 0.05909996034685324), (34, 0.04300951330909447), (35, 0.04849495735753291), (36, 0.04550805891878887), (37, 0.041

In [ ]:
print(get_recommendations2('Dune'))

[(0, 0.005292134719082175), (1, 0.007270929032170052), (2, 0.007753505332673432), (3, 0.005780966222427099), (4, 0.009148960359909716), (5, 0.007607602873388132), (6, 0.000954192114484824), (7, 0.0028281773031025477), (8, 0.0025266146353000877), (9, 0.004226378756236956), (10, 0.005278495169405), (11, 0.003058423714530229), (12, 0.006743247115976323), (13, 0.007519361807465719), (14, 0.006132317536146367), (15, 0.007869216388033945), (16, 0.0032078977539596702), (17, 0.002231158240045136), (18, 0.0096598794510082), (19, 0.010138608566765845), (20, 0.006342647116187816), (21, 0.0033685628985644146), (22, 0.006930708315453262), (23, 0.0042980330974468454), (24, 0.007006837109933028), (25, 0.0074655079134167855), (26, 0.0037072870746181195), (27, 0.008244483911209535), (28, 0.00684113262361294), (29, 0.0033735899718072824), (30, 0.0036513637088858267), (31, 0.002387833387205929), (32, 0.0072220666925868465), (33, 0.008376374000257649), (34, 0.0052014206719295224), (35, 0.00390609867737129

In [ ]:
print(get_recommendations2('A Fire Upon the Deep'))

[(0, 0.005920805227036372), (1, 0.007852483361976902), (2, 0.009498006536548973), (3, 0.012776660807839645), (4, 0.011915776248937623), (5, 1.0), (6, 0.0022362075875298044), (7, 0.002377960247453941), (8, 0.004222480433069307), (9, 0.009561790277228895), (10, 0.005515882938458077), (11, 0.005080368989131806), (12, 0.008805793950568717), (13, 0.007957477202210076), (14, 0.013316920064841772), (15, 0.011679690805844327), (16, 0.008055447708591307), (17, 0.003187752512542913), (18, 0.007352566910426508), (19, 0.01046602289131257), (20, 0.008740731487053765), (21, 0.019250656931875196), (22, 0.005552102008636867), (23, 0.011249535347001153), (24, 0.008697434711763853), (25, 0.010861394941655085), (26, 0.007461064733132125), (27, 0.012162785732357781), (28, 0.008588096618419971), (29, 0.009419127859562759), (30, 0.0028270687613425618), (31, 0.003248327265389787), (32, 0.013187109371645824), (33, 0.012960605468893651), (34, 0.005415550991972422), (35, 0.005065267804034406), (36, 0.0049149236